In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! git clone https://github.com/fra31/auto-attack.git

Cloning into 'auto-attack'...
remote: Enumerating objects: 672, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 672 (delta 162), reused 159 (delta 141), pack-reused 463
Receiving objects: 100% (672/672), 40.63 MiB | 35.81 MiB/s, done.
Resolving deltas: 100% (413/413), done.


In [3]:
!pip install git+https://github.com/fra31/auto-attack

  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-ddaohexh
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack /tmp/pip-req-build-ddaohexh
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py) ... done
  Created wheel for autoattack: filename=autoattack-0.1-py3-none-any.whl size=36229 sha256=be1dfccf479ddd64503babfbd8017e8ea9af54ca39d3cf63b1062434f37c4af9
  Stored in directory: /tmp/pip-ephem-wheel-cache-qhfakf69/wheels/b7/17/95/4b16fafe1b84fdabd247eb5f01b41165bd98d4c82d64ab93d2
Successfully built autoattack


In [4]:
# Copy Files
!cp /content/drive/MyDrive/ML-Project/Test3.rar /content

In [5]:
! unrar x /content/Test3.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /content/Test3.rar

Extracting  resnet.py                                                      5%  OK 
Extracting  ckpt.pth                                                      99%  OK 
All OK


In [6]:
!cp -r /content/resnet.py /content/auto-attack/autoattack/examples

In [7]:
%%shell
cat > /content/auto-attack/autoattack/examples/eval.py << EOF

import os
import argparse
from pathlib import Path
import warnings

import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms

import sys
sys.path.insert(0,'..')
from resnet import ResNet34

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='./data')
    parser.add_argument('--norm', type=str, default='L2')
    parser.add_argument('--epsilon', type=float, default=0.3)
    #///////////////////////////////////////////////////////////////
    parser.add_argument('--model', type=str, default='/content/ckpt.pth')
    #///////////////////////////////////////////////////////////////
    parser.add_argument('--n_ex', type=int, default=1000)
    parser.add_argument('--individual', action='store_true')
    parser.add_argument('--save_dir', type=str, default='./results')
    parser.add_argument('--batch_size', type=int, default=500)
    parser.add_argument('--log_path', type=str, default='./log_file.txt')
    parser.add_argument('--version', type=str, default='standard')
    parser.add_argument('--state-path', type=Path, default=None)

    args = parser.parse_args()

    # load model
    model = ResNet34()
    #//////////////////////////////////////////////
    model = nn.DataParallel(model)
    loaded = torch.load(args.model)
    model.load_state_dict(loaded['net'])
    #//////////////////////////////////////////////
    model.cuda()
    model.eval()

    # load data
    transform_list = [transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
    transform_chain = transforms.Compose(transform_list)
    item = datasets.CIFAR10(root=args.data_dir, train=False, transform=transform_chain, download=True)
    test_loader = data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)

    # create save dir
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    # load attack
    from autoattack import AutoAttack
    adversary = AutoAttack(model, norm=args.norm, eps=args.epsilon, log_path=args.log_path,
        version=args.version)

    l = [x for (x, y) in test_loader]
    x_test = torch.cat(l, 0)
    l = [y for (x, y) in test_loader]
    y_test = torch.cat(l, 0)

    # example of custom version
    if args.version == 'custom':
        adversary.attacks_to_run = ['apgd-ce', 'fab']
        adversary.apgd.n_restarts = 2
        adversary.fab.n_restarts = 2

    # run attack and save images
    with torch.no_grad():
        if not args.individual:
            adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
                bs=args.batch_size, state_path=args.state_path)

            torch.save({'adv_complete': adv_complete}, '{}/{}_{}_1_{}_eps_{:.5f}.pth'.format(
                args.save_dir, 'aa', args.version, adv_complete.shape[0], args.epsilon))

        else:
            # individual version, each attack is run on all test points
            adv_complete = adversary.run_standard_evaluation_individual(x_test[:args.n_ex],
                y_test[:args.n_ex], bs=args.batch_size)

            torch.save(adv_complete, '{}/{}_{}_individual_1_{}_eps_{:.5f}_plus_{}_cheap_{}.pth'.format(
                args.save_dir, 'aa', args.version, args.n_ex, args.epsilon))

/bin/bash: line 86: warning: here-document at line 0 delimited by end-of-file (wanted `EOF')


#AA-L2

##eps = 0.3

In [8]:
epsilon = 0.3
norm = 'L2'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

100% 170498071/170498071 [00:02<00:00, 82397083.10it/s]
Extracting ./data/cifar-10-python.tar.gz to ./data
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 10.30%
apgd-ce - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-CE: 10.30% (total time 8.2 s)
apgd-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-T: 10.30% (total time 82.4 s)
fab-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after FAB-T: 10.30% (total time 229.2 s)
Traceback (most recent call last):
  File "/content/auto-attack/autoattack/examples/eval.py", line 74, in <module>
    adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
  File "/usr/local/lib/python3.10/dist-packages/autoattack/autoattack.py", line 204, in run_standard_evaluation
    adv_curr = self.square.perturb(x, y)
  File "/usr/local/lib/python3.10/dist-packages/autoattack/square.py", line 59

##eps = 0.4

In [9]:
epsilon = 0.4
norm = 'L2'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 10.30%
apgd-ce - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-CE: 10.30% (total time 8.4 s)
apgd-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-T: 10.30% (total time 85.0 s)
fab-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after FAB-T: 10.30% (total time 232.9 s)
Traceback (most recent call last):
  File "/content/auto-attack/autoattack/examples/eval.py", line 74, in <module>
    adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
  File "/usr/local/lib/python3.10/dist-packages/autoattack/autoattack.py", line 204, in run_standard_evaluation
    adv_curr = self.square.perturb(x, y)
  File "/usr/local/lib/python3.10/dist-packages/autoattack/square.py", line 595, in perturb
    _, adv_curr = self.attack_single_run(x_to_fool, y_t

##eps = 0.5

In [10]:
epsilon = 0.5
norm = 'L2'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 10.30%
apgd-ce - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-CE: 10.30% (total time 8.4 s)
apgd-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-T: 10.30% (total time 85.2 s)
fab-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after FAB-T: 10.30% (total time 233.2 s)
Traceback (most recent call last):
  File "/content/auto-attack/autoattack/examples/eval.py", line 74, in <module>
    adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
  File "/usr/local/lib/python3.10/dist-packages/autoattack/autoattack.py", line 204, in run_standard_evaluation
    adv_curr = self.square.perturb(x, y)
  File "/usr/local/lib/python3.10/dist-packages/autoattack/square.py", line 595, in perturb
    _, adv_curr = self.attack_single_run(x_to_fool, y_t

#AA-Linf

##eps = 4/255

In [11]:
epsilon = 4/255
norm = 'Linf'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 10.30%
apgd-ce - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-CE: 10.30% (total time 8.4 s)
apgd-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-T: 10.30% (total time 84.9 s)
fab-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after FAB-T: 10.30% (total time 232.3 s)
Traceback (most recent call last):
  File "/content/auto-attack/autoattack/examples/eval.py", line 74, in <module>
    adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
  File "/usr/local/lib/python3.10/dist-packages/autoattack/autoattack.py", line 204, in run_standard_evaluation
    adv_curr = self.square.perturb(x, y)
  File "/usr/local/lib/python3.10/dist-packages/autoattack/square.py", line 595, in perturb
    _, adv_curr = self.attack_single_run(x_to_fool, y_t

##eps = 6/255

In [12]:
epsilon = 6/255
norm = 'Linf'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 10.30%
apgd-ce - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-CE: 10.30% (total time 8.4 s)
apgd-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-T: 10.30% (total time 85.0 s)
fab-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after FAB-T: 10.30% (total time 232.5 s)
Traceback (most recent call last):
  File "/content/auto-attack/autoattack/examples/eval.py", line 74, in <module>
    adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
  File "/usr/local/lib/python3.10/dist-packages/autoattack/autoattack.py", line 204, in run_standard_evaluation
    adv_curr = self.square.perturb(x, y)
  File "/usr/local/lib/python3.10/dist-packages/autoattack/square.py", line 595, in perturb
    _, adv_curr = self.attack_single_run(x_to_fool, y_t

##eps = 8/255

In [13]:
epsilon = 8/255
norm = 'Linf'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 10.30%
apgd-ce - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-CE: 10.30% (total time 8.4 s)
apgd-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after APGD-T: 10.30% (total time 85.0 s)
fab-t - 1/1 - 0 out of 103 successfully perturbed
robust accuracy after FAB-T: 10.30% (total time 232.4 s)
Traceback (most recent call last):
  File "/content/auto-attack/autoattack/examples/eval.py", line 74, in <module>
    adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
  File "/usr/local/lib/python3.10/dist-packages/autoattack/autoattack.py", line 204, in run_standard_evaluation
    adv_curr = self.square.perturb(x, y)
  File "/usr/local/lib/python3.10/dist-packages/autoattack/square.py", line 595, in perturb
    _, adv_curr = self.attack_single_run(x_to_fool, y_t